In [1]:
import pydicom, collections, csv, os, itertools

def read_tag_row(filename, foldername, filenumber, tags):
    ds = pydicom.dcmread(filename)
    list_to_return = [None] * (len(tags) + 2)
    list_to_return[0] = foldername # 'SCAN_ID = Folder name'
    list_to_return[1] = filenumber # '#_SLICE in series'
    for i in range(len(tags)):
        try:
            list_to_return[i+2] = ds[tags[i]][:]
        except TypeError:
            list_to_return[i+2] = str(ds[tags[i]].value)
        except KeyError:
            list_to_return[i+2] = ''
    return list_to_return

In [2]:
base_folder = '/workspace/CT/'
tags = [['0008','0070'], ['0008','103E'], ['0008','1090'], ['0010','0040'], ['0010','1010'], ['0018','0050'], ['0018','0060'], ['0018','1151'], ['0018','1210'], \
        ['0028','0010'], ['0028','0011'], ['0028','0030'], ['0018','0010']]
tag_names = ['SCAN_ID', '#_SLICE', 'Series Description', 'Manufacturer', ' Manufacturer\'s Model Name', 'Patient\'s Sex', 'Patient\'s Age', 'Slice Thickness', 'KVP', 'X-Ray Tube Current', 'Convolution Kernel', \
             'Rows', 'Columns', 'Pixel Spacing', 'Contrast/Bolus Agent']

f = open('output.csv', 'w')
wr = csv.writer(f)
wr.writerow(tag_names)
for dirpath, dirnames, filenames in os.walk(base_folder):
    for filename in itertools.islice(sorted([f for f in filenames if f.endswith(".dcm")]),1):
        file_abs = os.path.join(dirpath, filename)
        wr.writerow(read_tag_row(file_abs, os.path.basename(dirpath), len(filenames), tags))
f.close()